### OpenTargets Genetics Data Processing & Analysis Workflow

This notebook provides a reproducible workflow for batch processing, filtering, and analyzing OpenTargets Genetics datasets. It includes robust functions for:

- **Memory-efficient batch filtering** of large and small OpenTargets datasets using manifest-driven column selection.
- **Colocalization analysis**: Extracts high-confidence GWAS-QTL colocalizations, dynamically selecting available columns and providing comprehensive summary statistics.
- **Gene-drug-target mapping**: Integrates genetic loci, gene annotations, drug information, and disease associations to generate actionable gene-drug-target tables.

Each function is designed to handle missing columns gracefully, optimize memory usage, and output detailed diagnostics. The analyses support downstream interpretation of genetic evidence, variant-level colocalization, and drug target prioritization. This workflow is suitable for large-scale genetics projects and can be adapted to evolving OpenTargets data releases.

In [1]:
!pip install --upgrade pyarrow
!pip install duckdb
!pip install --upgrade pandas==2.2.2 numpy==2.0.2

In [2]:
import os
import json
import duckdb
import time
import gc
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.dataset as ds

from pathlib import Path

In [3]:
def process_single_dataset(dataset_name, columns, base_path, output_path):
    """
    Process a single Open Targets dataset by selecting specified columns and 
    writing a cleaned Parquet file to the output directory with memory-efficient handling.

    This function:
    - Checks if the dataset directory exists
    - Reads the dataset schema to validate requested columns
    - Streams large datasets (e.g., 'colocalisation_coloc', 'credible_set') in batches
      to prevent memory overflow
    - Saves a filtered version of the dataset with only valid columns

    Parameters
    ----------
    dataset_name : str
        Name of the dataset (e.g., 'study', 'l2g_prediction') to be processed.
    columns : list of str
        List of column names to extract from the dataset.
    base_path : str or Path
        Base directory path containing subdirectories for each dataset.
    output_path : str or Path
        Directory where the processed Parquet files will be saved.

    Returns
    -------
    bool
        True if processing was successful and the output file was written;
        False otherwise (e.g., directory missing, no valid columns, or error encountered).

    Notes
    -----
    - For very large datasets, batch processing is used to minimize memory usage.
    - Invalid columns (not present in schema) are skipped with a warning.
    - Requires `pyarrow.dataset` and `pyarrow.parquet`.
    - Performs garbage collection periodically to release memory.

    Examples
    --------
    >>> process_single_dataset("study", ["studyId", "geneId"], base_path, output_path)
    ✅ Saved study: 5,000 rows, 2 columns (1.2s)
    True
    """
    
    print(f"\n🔄 Processing {dataset_name}...")
    dataset_dir = os.path.join(base_path, dataset_name)
    
    try:
        # Check if directory exists
        if not os.path.exists(dataset_dir):
            print(f"⚠️  Directory does not exist: {dataset_dir}")
            return False
            
        start_time = time.time()
        
        # Only load valid parquet files
        dataset = ds.dataset(
            dataset_dir, 
            format="parquet", 
            exclude_invalid_files=True
        )
        
        # Get schema to check available columns
        schema = dataset.schema
        available_columns = [field.name for field in schema]
        print(f"📋 Available columns ({len(available_columns)}): {available_columns[:10]}{'...' if len(available_columns) > 10 else ''}")
        
        # Filter columns to only those that exist
        valid_columns = [col for col in columns if col in available_columns]
        missing_columns = [col for col in columns if col not in available_columns]
        
        if missing_columns:
            print(f"⚠️  Missing columns: {missing_columns}")
        
        if valid_columns:
            print(f"🔄 Converting to table with {len(valid_columns)} columns: {valid_columns}")
            
            # Process in chunks for large datasets to avoid memory issues
            if dataset_name in ['colocalisation_coloc', 'credible_set']:
                print(f"📊 Large dataset detected, using batch processing...")
                
                # For very large datasets, process in batches
                batch_size = 1000000  # 1M rows at a time
                batches = dataset.to_batches(columns=valid_columns, batch_size=batch_size)
                
                output_file = os.path.join(output_path, f"{dataset_name}.parquet")
                
                # Write first batch to create the file
                first_batch = next(batches)
                # Convert batch directly to table (no pandas conversion needed)
                table = pa.Table.from_batches([first_batch])
                
                parquet_writer = pq.ParquetWriter(output_file, table.schema)
                parquet_writer.write_table(table)
                
                total_rows = len(table)
                batch_count = 1
                
                # Process remaining batches
                for batch in batches:
                    # Convert batch directly to table
                    batch_table = pa.Table.from_batches([batch])
                    parquet_writer.write_table(batch_table)
                    total_rows += len(batch_table)
                    batch_count += 1
                    
                    if batch_count % 10 == 0:
                        print(f"  📊 Processed {batch_count} batches, {total_rows:,} rows so far...")
                        gc.collect()  # Force garbage collection
                
                parquet_writer.close()
                
            else:
                # For smaller datasets, process normally
                table = dataset.to_table(columns=valid_columns)
                output_file = os.path.join(output_path, f"{dataset_name}.parquet")
                pq.write_table(table, output_file)
                total_rows = len(table)
            
            elapsed = time.time() - start_time
            print(f"✅ Saved {dataset_name}: {total_rows:,} rows, {len(valid_columns)} columns ({elapsed:.1f}s)")
            
            # Force cleanup
            del dataset
            if 'table' in locals():
                del table
            gc.collect()
            
            return True
            
        else:
            print(f"❌ No valid columns found for {dataset_name}")
            return False
            
    except Exception as e:
        print(f"❌ Failed to process {dataset_name}: {e}")
        # Force cleanup on error
        gc.collect()
        return False

In [4]:
def analyze_gwas_colocalizations_with_available_columns(output_path: str) -> pd.DataFrame:
    """
    Analyze GWAS-QTL colocalizations with dynamic column selection and comprehensive statistics.
    
    This function performs a robust analysis of genetic colocalization data by first checking
    what columns are available in the filtered OpenTargets datasets, then building a dynamic
    SQL query to extract maximum information. It focuses on high-confidence colocalizations
    (H4 > 0.8) and provides detailed statistical summaries.
    
    Parameters
    ----------
    output_path : str
        Path to directory containing filtered OpenTargets parquet files. Expected files:
        - colocalisation_coloc.parquet : Colocalization evidence between study pairs
        - credible_set.parquet : Fine-mapped credible sets for study loci  
        - study.parquet : Study metadata including traits and types
        
    Returns
    -------
    pd.DataFrame or None
        DataFrame containing colocalization analysis with available columns, or None if failed.
        
        Core columns (always present):
        - leftStudyLocusId : Study-locus identifier for GWAS signal
        - rightStudyLocusId : Study-locus identifier for QTL signal
        - studyId : Study identifier from credible set
        - chromosome : Chromosomal location of colocalization
        
        Optional columns (included if available in source data):
        - numberColocalisingVariants : Number of variants supporting colocalization
        - variantId : Lead variant identifier (rs ID or chr:pos format)
        - isTransQtl : Boolean indicating trans-QTL (distant) vs cis-QTL (local)
        - h0, h1, h2, h3, h4 : Posterior probabilities for colocalization hypotheses
        - traitFromSource : Human-readable trait description
        - studyType : Type of left-side study (gwas, eqtl, pqtl, etc.)
        - rightStudyType : Type of right-side study (eqtl, pqtl, etc.)
        
    Algorithm
    ---------
    1. **Schema Detection**: Checks available columns in each parquet file
    2. **Dynamic Query Building**: Constructs SQL with only available columns
    3. **High-Confidence Filtering**: Selects colocalizations with H4 > 0.8
    4. **Multi-table Join**: Links colocalization → credible set → study metadata
    5. **Statistical Analysis**: Computes comprehensive summaries by column type
    
    Query Logic
    -----------
    ```sql
    SELECT [dynamic_columns]
    FROM colocalisation_coloc (H4 > 0.8, sample 10K rows)
    LEFT JOIN credible_set ON rightStudyLocusId = studyLocusId  
    LEFT JOIN study ON credible_set.studyId = study.studyId
    WHERE studyId IS NOT NULL
    ORDER BY H4 DESC LIMIT 100
    ```
    
    Statistical Outputs
    ------------------
    - **Data Completeness**: Missing value analysis for each column
    - **Variant Analysis**: Statistics on numberColocalisingVariants (mean, median, range)
    - **Variant ID Analysis**: Uniqueness and example variant identifiers
    - **Trans-QTL Analysis**: Proportion of trans vs cis regulatory effects
    - **Colocalization Evidence**: Mean posterior probabilities (H0-H4)
    - **Study Type Distribution**: Breakdown by GWAS, eQTL, pQTL types
    - **Cross-tabulations**: Variants by study type combinations
    
    Colocalization Hypotheses (H-values)
    ------------------------------------
    - H0: Neither trait has genetic association at locus
    - H1: Only left trait (typically GWAS) has association  
    - H2: Only right trait (typically QTL) has association
    - H3: Both traits associated, but different causal variants
    - H4: Both traits associated, shared causal variant (colocalization)
    
    Memory Management
    ----------------
    - Uses DuckDB for efficient parquet processing
    - Samples large datasets (10K rows) to prevent memory issues
    - Automatic connection cleanup via context manager
    
    Error Handling
    --------------
    - Graceful handling of missing parquet files
    - Column availability checking prevents SQL errors
    - Detailed error reporting with query debugging information
    
    Examples
    --------
    >>> # Basic usage
    >>> df = analyze_gwas_colocalizations_with_available_columns('/path/to/filtered_data')
    >>> print(f"Found {len(df)} high-confidence colocalizations")
    
    >>> # Check for specific analyses
    >>> if 'isTransQtl' in df.columns:
    >>>     trans_qtl_pct = df['isTransQtl'].mean() * 100
    >>>     print(f"Trans-QTL percentage: {trans_qtl_pct:.1f}%")
    
    >>> # Filter by study type
    >>> eqtl_colocs = df[df['rightStudyType'] == 'eqtl']
    >>> print(f"eQTL colocalizations: {len(eqtl_colocs)}")
    
    Notes
    -----
    - Function prints extensive diagnostic information during execution
    - Designed to work with any subset of OpenTargets columns
    - H4 > 0.8 threshold represents high-confidence shared causal variants
    - Sample size (10K rows) balances comprehensiveness with performance
    - Results ordered by H4 (highest confidence first)
    
    See Also
    --------
    - OpenTargets Genetics documentation: https://genetics-docs.opentargets.org/
    - Colocalization methods: Giambartolomei et al. (2014) PLoS Genet
    - COLOC R package: https://github.com/chr1swallace/coloc
    """
    
    con = duckdb.connect()
    
    print("🔍 Checking what columns are actually available in your filtered datasets...")
    
    # Check available columns in each table
    tables_info = {}
    
    for table_name in ['colocalisation_coloc', 'credible_set', 'study']:
        try:
            query = f"SELECT * FROM read_parquet('{output_path}/{table_name}.parquet') LIMIT 1"
            sample = con.execute(query).df()
            tables_info[table_name] = sample.columns.tolist()
            print(f"  📋 {table_name}: {sample.columns.tolist()}")
        except Exception as e:
            print(f"  ❌ {table_name}: Error - {e}")
            tables_info[table_name] = []
    
    # Build query with only available columns
    print(f"\n🔧 Building query with available columns...")
    
    # Core columns that should always be there
    select_columns = [
        "coloc.leftStudyLocusId",
        "coloc.rightStudyLocusId",
        "credible.studyId", 
        "coloc.chromosome"
    ]
    
    
    # Add h-values if available
    h_columns = ['h0', 'h1', 'h2', 'h3', 'h4']
    available_h_cols = []
    for h_col in h_columns:
        if h_col in tables_info.get('colocalisation_coloc', []):
            select_columns.append(f"coloc.{h_col}")
            available_h_cols.append(h_col)
    
    print(f"  ✅ Available h-columns: {available_h_cols}")
    
    # Add study columns if available
    study_cols = ['traitFromSource', 'studyType']
    available_study_cols = []
    for col in study_cols:
        if col in tables_info.get('study', []):
            select_columns.append(f"study.{col}")
            available_study_cols.append(col)
    
    print(f"  ✅ Available study columns: {available_study_cols}")

    # Add numberColocalisingVariants if available
    if 'numberColocalisingVariants' in tables_info.get('colocalisation_coloc', []):
        select_columns.append("coloc.numberColocalisingVariants")
        print(f"  ✅ numberColocalisingVariants: Available")
    else:
        print(f"  ❌ numberColocalisingVariants: Not available in your filtered dataset")
    
    # Add variantId if available
    if 'variantId' in tables_info.get('credible_set', []):
        select_columns.append("credible.variantId")
        print(f"  ✅ variantId: Available")
    else:
        print(f"  ❌ variantId: Not available in your filtered dataset")

    # Add isTransQtl if available
    if 'isTransQtl' in tables_info.get('credible_set', []):
        select_columns.append("credible.isTransQtl")
        print(f"  ✅ isTransQtl: Available")
    else:
        print(f"  ❌ isTransQtl: Not available in your filtered dataset")
    
    # Check for rightStudyType specifically
    if 'rightStudyType' in tables_info.get('colocalisation_coloc', []):
        select_columns.append("coloc.rightStudyType")
        print(f"  ✅ rightStudyType: Available")
    else:
        print(f"  ❌ rightStudyType: Not available in your filtered dataset")
    
    # Build the final query
    select_clause = ",\n      ".join(select_columns)
    
    specific_query = f"""
    SELECT 
    {select_clause}                                    -- Dynamic column selection based on available data
    FROM (
    -- Step 1: Filter high-confidence colocalizations from main dataset
    SELECT * FROM read_parquet('{output_path}/colocalisation_coloc.parquet')
    WHERE h4 > 0.8                                     -- Only include strong colocalization evidence (>80% probability)
    USING SAMPLE 10000 ROWS                            -- Random sample to prevent memory issues with large datasets
    ) coloc
    -- Step 2: Join with credible sets to get variant-level information
    LEFT JOIN read_parquet('{output_path}/credible_set.parquet') credible
    ON coloc.rightStudyLocusId = credible.studyLocusId  -- Link colocalization to fine-mapped variants
    -- Step 3: Join with study metadata to get trait and publication information  
    LEFT JOIN read_parquet('{output_path}/study.parquet') study
    ON credible.studyId = study.studyId                  -- Link credible sets to study descriptions
    WHERE credible.studyId IS NOT NULL                   -- Filter out colocalizations without credible set matches
    ORDER BY coloc.h4 DESC                               -- Sort by colocalization confidence (highest first)
    LIMIT 100                                            -- Return top 100 results for manageable analysis
    """
    
    print(f"\n🚀 Executing query with {len(select_columns)} available columns...")
    print(f"📋 Selected columns: {[col.split('.')[-1] for col in select_columns]}")
    
    try:
        df_specific = con.execute(specific_query).df()
        print(f"✅ Dataset created: {len(df_specific)} rows")
        print(f"📋 Final columns: {df_specific.columns.tolist()}")
        
        # Data completeness analysis
        print(f"\n📊 Data completeness:")
        for col in df_specific.columns:
            non_null = df_specific[col].notna().sum()
            print(f"  {col}: {non_null}/{len(df_specific)} ({non_null/len(df_specific)*100:.1f}%)")
        
        # Show sample data
        print(f"\n👀 Sample of your dataset:")
        print(df_specific.head())
        
        # Analyze numberColocalisingVariants if available
        if 'numberColocalisingVariants' in df_specific.columns:
            print(f"\n🔗 Colocalising Variants Analysis:")
            variants_stats = df_specific['numberColocalisingVariants'].describe()
            print(f"  Mean variants per colocalization: {variants_stats['mean']:.1f}")
            print(f"  Median variants: {variants_stats['50%']:.1f}")
            print(f"  Range: {variants_stats['min']:.0f} - {variants_stats['max']:.0f}")
            print(f"  High variant count (>10): {(df_specific['numberColocalisingVariants'] > 10).sum()} pairs")
        
        # Analyze variantId if available
        if 'variantId' in df_specific.columns:
            print(f"\n🧬 Variant ID Analysis:")
            unique_variants = df_specific['variantId'].nunique()
            total_rows = len(df_specific)
            print(f"  Unique variants: {unique_variants:,}")
            print(f"  Total rows: {total_rows:,}")
            print(f"  Variants per row ratio: {unique_variants/total_rows:.2f}")
            
            # Show some example variant IDs
            sample_variants = df_specific['variantId'].dropna().head().tolist()
            print(f"  Example variant IDs: {sample_variants}")
        
        # Analyze isTransQtl if available
        if 'isTransQtl' in df_specific.columns:
            print(f"\n🧬 Trans-QTL Analysis:")
            trans_qtl_count = df_specific['isTransQtl'].sum()
            total_rows = len(df_specific)
            print(f"  Trans-QTL colocalizations: {trans_qtl_count}/{total_rows} ({trans_qtl_count/total_rows*100:.1f}%)")
            print(f"  Cis-QTL colocalizations: {total_rows - trans_qtl_count}/{total_rows} ({(total_rows - trans_qtl_count)/total_rows*100:.1f}%)")
        
        # Analyze colocalization evidence if h-values available
        if available_h_cols:
            print(f"\n🧬 Colocalization analysis:")
            print(f"  Available evidence: {available_h_cols}")
            print(f"  Mean posterior probabilities:")
            for h_col in available_h_cols:
                if h_col in df_specific.columns:
                    mean_val = df_specific[h_col].mean()
                    print(f"    {h_col}: {mean_val:.4f}")
        
        # Study type analysis if available
        if 'studyType' in df_specific.columns:
            print(f"\n📊 Study type distribution:")
            study_counts = df_specific['studyType'].value_counts()
            for study_type, count in study_counts.items():
                print(f"  {study_type}: {count} rows")
        
        # Right study type analysis if available
        if 'rightStudyType' in df_specific.columns:
            print(f"\n📊 Right study type distribution:")
            right_study_counts = df_specific['rightStudyType'].value_counts()
            for study_type, count in right_study_counts.items():
                print(f"  {study_type}: {count} rows")
        
        # Cross-analysis if both variants and study types are available
        if 'numberColocalisingVariants' in df_specific.columns and 'rightStudyType' in df_specific.columns:
            print(f"\n🔗 Variants by right study type:")
            variants_by_type = df_specific.groupby('rightStudyType')['numberColocalisingVariants'].agg(['mean', 'count'])
            for study_type, stats in variants_by_type.iterrows():
                print(f"  {study_type}: {stats['mean']:.1f} avg variants (n={stats['count']})")
        
        return df_specific
        
    except Exception as e:
        print(f"❌ Query failed: {e}")
        print(f"\nQuery was:")
        print(specific_query)
        return None
    finally:
        con.close()

In [5]:
### FUNCTION FROM MAURO
def analyze_gwas_colocalizations_with_available_columns_doubleJoin(output_path: str) -> pd.DataFrame:
    """
    Analyze GWAS-QTL colocalizations with dynamic column selection and comprehensive statistics.
    
    This function performs a robust analysis of genetic colocalization data by first checking
    what columns are available in the filtered OpenTargets datasets, then building a dynamic
    SQL query to extract maximum information. It focuses on high-confidence colocalizations
    (H4 > 0.8) and provides detailed statistical summaries.
    
    Parameters
    ----------
    output_path : str
        Path to directory containing filtered OpenTargets parquet files. Expected files:
        - colocalisation_coloc.parquet : Colocalization evidence between study pairs
        - credible_set.parquet : Fine-mapped credible sets for study loci  
        - study.parquet : Study metadata including traits and types
        
    Returns
    -------
    pd.DataFrame or None
        DataFrame containing colocalization analysis with available columns, or None if failed.
        
        Core columns (always present):
        - leftStudyLocusId : Study-locus identifier for GWAS signal
        - rightStudyLocusId : Study-locus identifier for QTL signal
        - studyId : Study identifier from credible set
        - chromosome : Chromosomal location of colocalization
        
        Optional columns (included if available in source data):
        - numberColocalisingVariants : Number of variants supporting colocalization
        - variantId : Lead variant identifier (rs ID or chr:pos format)
        - isTransQtl : Boolean indicating trans-QTL (distant) vs cis-QTL (local)
        - h0, h1, h2, h3, h4 : Posterior probabilities for colocalization hypotheses
        - traitFromSource : Human-readable trait description
        - studyType : Type of left-side study (gwas, eqtl, pqtl, etc.)
        - rightStudyType : Type of right-side study (eqtl, pqtl, etc.)
        
    Algorithm
    ---------
    1. **Schema Detection**: Checks available columns in each parquet file
    2. **Dynamic Query Building**: Constructs SQL with only available columns
    3. **High-Confidence Filtering**: Selects colocalizations with H4 > 0.8
    4. **Multi-table Join**: Links colocalization → credible set → study metadata
    5. **Statistical Analysis**: Computes comprehensive summaries by column type
    
    Query Logic
    -----------
    ```sql
    SELECT [dynamic_columns],
       cs_right.studyLocusId   AS studyLocusId_right,
       s_right.studyId         AS studyId_right,
       s_right.traitReported   AS traitReported_right,
       s_right.pubAuthor       AS pubAuthor_right,
       s_right.pubJournal      AS pubJournal_right,
       s_right.pubYear         AS pubYear_right,
       cs_left.studyLocusId    AS studyLocusId_left,
       s_left.studyId          AS studyId_left,
       s_left.traitReported    AS traitReported_left,
       s_left.pubAuthor        AS pubAuthor_left,
       s_left.pubJournal       AS pubJournal_left,
       s_left.pubYear          AS pubYear_left
    FROM colocalisation_coloc c
         LEFT JOIN credible_set cs_right ON c.rightStudyLocusId = cs_right.studyLocusId
         LEFT JOIN study s_right         ON cs_right.studyId = s_right.studyId
         LEFT JOIN credible_set cs_left  ON c.leftStudyLocusId = cs_left.studyLocusId
         LEFT JOIN study s_left          ON cs_left.studyId = s_left.studyId
    WHERE (s_right.studyId IS NOT NULL OR s_left.studyId IS NOT NULL)
      AND c.H4 > 0.8 
    ORDER BY c.H4 DESC
    LIMIT 100;
    ```
    
    Statistical Outputs
    ------------------
    - **Data Completeness**: Missing value analysis for each column
    - **Variant Analysis**: Statistics on numberColocalisingVariants (mean, median, range)
    - **Variant ID Analysis**: Uniqueness and example variant identifiers
    - **Trans-QTL Analysis**: Proportion of trans vs cis regulatory effects
    - **Colocalization Evidence**: Mean posterior probabilities (H0-H4)
    - **Study Type Distribution**: Breakdown by GWAS, eQTL, pQTL types
    - **Cross-tabulations**: Variants by study type combinations
    
    Colocalization Hypotheses (H-values)
    ------------------------------------
    - H0: Neither trait has genetic association at locus
    - H1: Only left trait (typically GWAS) has association  
    - H2: Only right trait (typically QTL) has association
    - H3: Both traits associated, but different causal variants
    - H4: Both traits associated, shared causal variant (colocalization)
    
    Memory Management
    ----------------
    - Uses DuckDB for efficient parquet processing
    - Samples large datasets (10K rows) to prevent memory issues
    - Automatic connection cleanup via context manager
    
    Error Handling
    --------------
    - Graceful handling of missing parquet files
    - Column availability checking prevents SQL errors
    - Detailed error reporting with query debugging information
    
    Examples
    --------
    >>> # Basic usage
    >>> df = analyze_gwas_colocalizations_with_available_columns('/path/to/filtered_data')
    >>> print(f"Found {len(df)} high-confidence colocalizations")
    
    >>> # Check for specific analyses
    >>> if 'isTransQtl' in df.columns:
    >>>     trans_qtl_pct = df['isTransQtl'].mean() * 100
    >>>     print(f"Trans-QTL percentage: {trans_qtl_pct:.1f}%")
    
    >>> # Filter by study type
    >>> eqtl_colocs = df[df['rightStudyType'] == 'eqtl']
    >>> print(f"eQTL colocalizations: {len(eqtl_colocs)}")
    
    Notes
    -----
    - Function prints extensive diagnostic information during execution
    - Designed to work with any subset of OpenTargets columns
    - H4 > 0.8 threshold represents high-confidence shared causal variants
    - Sample size (10K rows) balances comprehensiveness with performance
    - Results ordered by H4 (highest confidence first)
    
    See Also
    --------
    - OpenTargets Genetics documentation: https://genetics-docs.opentargets.org/
    - Colocalization methods: Giambartolomei et al. (2014) PLoS Genet
    - COLOC R package: https://github.com/chr1swallace/coloc
    """
    
    con = duckdb.connect()
    
    print("🔍 Checking what columns are actually available in your filtered datasets...")
    
    # Check available columns in each table
    tables_info = {}
    
    for table_name in ['colocalisation_coloc', 'credible_set', 'study']:
        try:
            query = f"SELECT * FROM read_parquet('{output_path}/{table_name}.parquet') LIMIT 1"
            sample = con.execute(query).df()
            tables_info[table_name] = sample.columns.tolist()
            print(f"  📋 {table_name}: {sample.columns.tolist()}")
        except Exception as e:
            print(f"  ❌ {table_name}: Error - {e}")
            tables_info[table_name] = []
    
       # Core columns that should always be there
    select_columns = [
        "coloc.leftStudyLocusId",
        "coloc.rightStudyLocusId",
        "credible_right.studyId AS studyId_right", 
        "credible_left.studyId AS studyId_left",
        "coloc.chromosome"
    ]
    
    # Add h-values if available
    h_columns = ['h0', 'h1', 'h2', 'h3', 'h4']
    for h_col in h_columns:
        if h_col in tables_info.get('colocalisation_coloc', []):
            select_columns.append(f"coloc.{h_col}")
    
    # Add right-side study metadata
    study_cols = ['traitFromSource', 'studyType']
    for col in study_cols:
        if col in tables_info.get('study', []):
            select_columns.append(f"study_right.{col} AS {col}_right")
            select_columns.append(f"study_left.{col} AS {col}_left")
    
    # Add numberColocalisingVariants if available
    if 'numberColocalisingVariants' in tables_info.get('colocalisation_coloc', []):
        select_columns.append("coloc.numberColocalisingVariants")
    
    # Add variantId if available
    if 'variantId' in tables_info.get('credible_set', []):
        select_columns.append("credible_right.variantId AS variantId_right")
        select_columns.append("credible_left.variantId AS variantId_left")
    
    # Add isTransQtl if available
    if 'isTransQtl' in tables_info.get('credible_set', []):
        select_columns.append("credible_right.isTransQtl AS isTransQtl_right")
        select_columns.append("credible_left.isTransQtl AS isTransQtl_left")
    
    # Add rightStudyType if available
    if 'rightStudyType' in tables_info.get('colocalisation_coloc', []):
        select_columns.append("coloc.rightStudyType")
    
    # Build the final query
    select_clause = ",\n      ".join(select_columns)
    
    specific_query = f"""
    SELECT 
        {select_clause}
    FROM (
        SELECT * FROM read_parquet('{output_path}/colocalisation_coloc.parquet')
        WHERE h4 > 0.8 
            AND rightStudyType <> 'gwas'                     -- 🚫 Exclude GWAS on right side (pre-filter!)
        USING SAMPLE 10000 ROWS
    ) coloc
    -- Right-side join
    LEFT JOIN read_parquet('{output_path}/credible_set.parquet') credible_right
        ON coloc.rightStudyLocusId = credible_right.studyLocusId
    LEFT JOIN read_parquet('{output_path}/study.parquet') study_right
        ON credible_right.studyId = study_right.studyId
    -- Left-side join
    LEFT JOIN read_parquet('{output_path}/credible_set.parquet') credible_left
        ON coloc.leftStudyLocusId = credible_left.studyLocusId
    LEFT JOIN read_parquet('{output_path}/study.parquet') study_left
        ON credible_left.studyId = study_left.studyId
    WHERE credible_right.studyId IS NOT NULL 
       OR credible_left.studyId IS NOT NULL
       OR coloc.rightStudyType <> 'gwas'  
    ORDER BY coloc.h4 DESC
    LIMIT 100
    """
    
    print(f"\n🚀 Executing query with {len(select_columns)} available columns...")
    print(f"📋 Selected columns: {[col.split('.')[-1] for col in select_columns]}")
    
    try:
        df_specific = con.execute(specific_query).df()
        print(f"✅ Dataset created: {len(df_specific)} rows")
        print(f"📋 Final columns: {df_specific.columns.tolist()}")
        
        # Data completeness analysis
        print(f"\n📊 Data completeness:")
        for col in df_specific.columns:
            non_null = df_specific[col].notna().sum()
            print(f"  {col}: {non_null}/{len(df_specific)} ({non_null/len(df_specific)*100:.1f}%)")
        
        # Show sample data
        print(f"\n👀 Sample of your dataset:")
        print(df_specific.head())
        
        # Analyze numberColocalisingVariants if available
        if 'numberColocalisingVariants' in df_specific.columns:
            print(f"\n🔗 Colocalising Variants Analysis:")
            variants_stats = df_specific['numberColocalisingVariants'].describe()
            print(f"  Mean variants per colocalization: {variants_stats['mean']:.1f}")
            print(f"  Median variants: {variants_stats['50%']:.1f}")
            print(f"  Range: {variants_stats['min']:.0f} - {variants_stats['max']:.0f}")
            print(f"  High variant count (>10): {(df_specific['numberColocalisingVariants'] > 10).sum()} pairs")
        
        # Analyze variantId if available
        if 'variantId' in df_specific.columns:
            print(f"\n🧬 Variant ID Analysis:")
            unique_variants = df_specific['variantId'].nunique()
            total_rows = len(df_specific)
            print(f"  Unique variants: {unique_variants:,}")
            print(f"  Total rows: {total_rows:,}")
            print(f"  Variants per row ratio: {unique_variants/total_rows:.2f}")
            
            # Show some example variant IDs
            sample_variants = df_specific['variantId'].dropna().head().tolist()
            print(f"  Example variant IDs: {sample_variants}")
        
        # Analyze isTransQtl if available
        if 'isTransQtl' in df_specific.columns:
            print(f"\n🧬 Trans-QTL Analysis:")
            trans_qtl_count = df_specific['isTransQtl'].sum()
            total_rows = len(df_specific)
            print(f"  Trans-QTL colocalizations: {trans_qtl_count}/{total_rows} ({trans_qtl_count/total_rows*100:.1f}%)")
            print(f"  Cis-QTL colocalizations: {total_rows - trans_qtl_count}/{total_rows} ({(total_rows - trans_qtl_count)/total_rows*100:.1f}%)")
        
        # Analyze colocalization evidence if h-values available
       # if available_h_cols:
        #    print(f"\n🧬 Colocalization analysis:")
       #     print(f"  Available evidence: {available_h_cols}")
        #    print(f"  Mean posterior probabilities:")
        #    for h_col in available_h_cols:
        #        if h_col in df_specific.columns:
        #            mean_val = df_specific[h_col].mean()
        #            print(f"    {h_col}: {mean_val:.4f}")
        
        # Study type analysis if available
        if 'studyType' in df_specific.columns:
            print(f"\n📊 Study type distribution:")
            study_counts = df_specific['studyType'].value_counts()
            for study_type, count in study_counts.items():
                print(f"  {study_type}: {count} rows")
        
        # Right study type analysis if available
        if 'rightStudyType' in df_specific.columns:
            print(f"\n📊 Right study type distribution:")
            right_study_counts = df_specific['rightStudyType'].value_counts()
            for study_type, count in right_study_counts.items():
                print(f"  {study_type}: {count} rows")
        
        # Cross-analysis if both variants and study types are available
        if 'numberColocalisingVariants' in df_specific.columns and 'rightStudyType' in df_specific.columns:
            print(f"\n🔗 Variants by right study type:")
            variants_by_type = df_specific.groupby('rightStudyType')['numberColocalisingVariants'].agg(['mean', 'count'])
            for study_type, stats in variants_by_type.iterrows():
                print(f"  {study_type}: {stats['mean']:.1f} avg variants (n={stats['count']})")
        
        return df_specific
        
    except Exception as e:
        print(f"❌ Query failed: {e}")
        print(f"\nQuery was:")
        print(specific_query)
        return None
    finally:
        con.close()

In [6]:
def create_gene_drug_target_dataframe(output_path: str) -> pd.DataFrame:
    """
    Create a dataframe linking genes to drug targets using OpenTargets data.
    
    This function combines multiple datasets to create comprehensive gene-drug-target mappings:
    - l2g_prediction: Links genetic loci to genes (geneId)
    - evidence: Links targets to diseases (targetId) 
    - known_drug: Links drugs to targets (drugId, targetId)
    - target: Provides gene symbols and annotations
    
    Parameters
    ----------
    output_path : str
        Path to directory containing filtered OpenTargets parquet files
        
    Returns
    -------
    pd.DataFrame
        DataFrame with gene and drug target information
    """
    
    con = duckdb.connect()
    
    print("🔍 Checking available datasets for gene-drug-target mapping...")
    
    # Check available columns in each relevant table
    tables_info = {}
    required_tables = ['l2g_prediction', 'evidence', 'known_drug', 'target']
    
    for table_name in required_tables:
        try:
            query = f"SELECT * FROM read_parquet('{output_path}/{table_name}.parquet') LIMIT 1"
            sample = con.execute(query).df()
            tables_info[table_name] = sample.columns.tolist()
            print(f"  📋 {table_name}: {sample.columns.tolist()}")
        except Exception as e:
            print(f"  ❌ {table_name}: Error - {e}")
            tables_info[table_name] = []
    
    # Build dynamic query based on available columns
    print(f"\n🔧 Building gene-drug-target query...")
    
    # Core columns we want to extract
    select_columns = []
    
    # From l2g_prediction: geneId, studyLocusId, score
    if 'geneId' in tables_info.get('l2g_prediction', []):
        select_columns.extend([
            "l2g.geneId",
            "l2g.studyLocusId", 
            "l2g.score as l2g_score"
        ])
        print(f"  ✅ l2g_prediction: geneId available")
    
    # From evidence: targetId, diseaseId
    if 'targetId' in tables_info.get('evidence', []):
        select_columns.extend([
            "evidence.targetId",
            "evidence.diseaseId"
        ])
        print(f"  ✅ evidence: targetId available")
    
    # From known_drug: drugId and available columns (check what actually exists)
    known_drug_cols = []
    if 'drugId' in tables_info.get('known_drug', []):
        known_drug_cols.append("drug.drugId")
        
        # Check for optional columns that might exist
        if 'phase' in tables_info.get('known_drug', []):
            known_drug_cols.append("drug.phase")
        if 'status' in tables_info.get('known_drug', []):
            known_drug_cols.append("drug.status")
            
        select_columns.extend(known_drug_cols)
        print(f"  ✅ known_drug: {[col.split('.')[-1] for col in known_drug_cols]} available")
    
    # From target: gene symbol and annotations
    target_cols = []
    if 'approvedSymbol' in tables_info.get('target', []):
        target_cols.append("target.approvedSymbol as gene_symbol")
    if 'biotype' in tables_info.get('target', []):
        target_cols.append("target.biotype")
    
    if target_cols:
        select_columns.extend(target_cols)
        print(f"  ✅ target: {[col.split('.')[-1].split(' as ')[-1] for col in target_cols]} available")
    
    if not select_columns:
        print("❌ No required columns found in datasets")
        return None
    
    # Build the comprehensive query
    select_clause = ",\n      ".join(select_columns)
    
    gene_drug_query = f"""
    SELECT 
      {select_clause}
    FROM (
      -- Start with locus-to-gene predictions (high confidence genes)
      SELECT * FROM read_parquet('{output_path}/l2g_prediction.parquet')
      WHERE score > 0.5  -- Only include high-confidence gene predictions
      USING SAMPLE 50000 ROWS  -- Sample to manage memory
    ) l2g
    -- Join with evidence to get target-disease associations
    LEFT JOIN read_parquet('{output_path}/evidence.parquet') evidence
      ON l2g.geneId = evidence.targetId  -- Gene ID = Target ID in OpenTargets
    -- Join with known drugs to get drug information
    LEFT JOIN read_parquet('{output_path}/known_drug.parquet') drug
      ON evidence.targetId = drug.targetId 
      AND evidence.diseaseId = drug.diseaseId  -- Match target-disease pairs
    -- Join with target info to get gene symbols
    LEFT JOIN read_parquet('{output_path}/target.parquet') target
      ON l2g.geneId = target.id  -- Gene ID matches target ID
    WHERE evidence.targetId IS NOT NULL  -- Ensure we have target information
      AND drug.drugId IS NOT NULL        -- Ensure we have drug information
    ORDER BY l2g.score DESC  -- Prioritize high-confidence genes
    LIMIT 1000  -- Return top 1000 gene-drug-target associations
    """
    
    print(f"\n🚀 Executing gene-drug-target query...")
    print(f"📋 Selected columns: {[col.split('.')[-1].split(' as ')[-1] for col in select_columns]}")
    
    try:
        df_gene_drug = con.execute(gene_drug_query).df()
        print(f"✅ Dataset created: {len(df_gene_drug)} rows")
        print(f"📋 Final columns: {df_gene_drug.columns.tolist()}")
        
        # Data summary
        print(f"\n📊 Gene-Drug-Target Summary:")
        if 'geneId' in df_gene_drug.columns:
            unique_genes = df_gene_drug['geneId'].nunique()
            print(f"  Unique genes: {unique_genes:,}")
        
        if 'drugId' in df_gene_drug.columns:
            unique_drugs = df_gene_drug['drugId'].nunique()
            print(f"  Unique drugs: {unique_drugs:,}")
        
        if 'targetId' in df_gene_drug.columns:
            unique_targets = df_gene_drug['targetId'].nunique()
            print(f"  Unique targets: {unique_targets:,}")
        
        # Clinical phase analysis (only if phase column exists)
        if 'phase' in df_gene_drug.columns:
            print(f"\n💊 Drug Development Phases:")
            phase_counts = df_gene_drug['phase'].value_counts().sort_index()
            for phase, count in phase_counts.items():
                phase_name = {0: "Preclinical", 1: "Phase I", 2: "Phase II", 
                            3: "Phase III", 4: "Approved"}.get(phase, f"Phase {phase}")
                print(f"  {phase_name}: {count} associations")
        
        # L2G score analysis
        if 'l2g_score' in df_gene_drug.columns:
            print(f"\n🎯 Locus-to-Gene Score Distribution:")
            score_stats = df_gene_drug['l2g_score'].describe()
            print(f"  Mean score: {score_stats['mean']:.3f}")
            print(f"  High confidence (>0.8): {(df_gene_drug['l2g_score'] > 0.8).sum()} genes")
        
        # Show sample data
        print(f"\n👀 Sample gene-drug-target associations:")
        display_cols = ['geneId', 'gene_symbol', 'drugId', 'targetId', 'l2g_score']
        if 'phase' in df_gene_drug.columns:
            display_cols.insert(-1, 'phase')
        available_display_cols = [col for col in display_cols if col in df_gene_drug.columns]
        print(df_gene_drug[available_display_cols].head())
        
        return df_gene_drug
        
    except Exception as e:
        print(f"❌ Query failed: {e}")
        print(f"\nQuery was:")
        print(gene_drug_query)
        return None
    finally:
        con.close()

# Usage example - make sure to define the paths
def run_gene_drug_analysis():
    """Run the gene-drug-target analysis and save results."""
    
    print("🚀 Creating gene-drug-target dataframe...")
    df_gene_drug = create_gene_drug_target_dataframe(output_path)
    
    if df_gene_drug is not None:
        print(f"\n🎉 Analysis complete!")
        print(f"📊 Generated {len(df_gene_drug)} gene-drug-target associations")
        
        # Save the dataset
        output_file = main_dir / "gene_drug_target_associations.csv"
        df_gene_drug.to_csv(output_file, index=False)
        print(f"💾 Saved to: {output_file}")
        
        # Additional analysis
        print(f"\n🔬 Key Insights:")
        if 'gene_symbol' in df_gene_drug.columns and 'drugId' in df_gene_drug.columns:
            # Top genes by drug count
            top_genes = df_gene_drug.groupby('gene_symbol')['drugId'].nunique().sort_values(ascending=False).head()
            print(f"  Top genes by drug count:")
            for gene, drug_count in top_genes.items():
                print(f"    {gene}: {drug_count} drugs")
        
        return df_gene_drug
    else:
        print("❌ Failed to create gene-drug-target dataframe")
        return None

In [7]:
main_dir = Path("/ictstr01/groups/itg/shared/referenceData/OpenTargets/Colocalizations/envs/")

base_path = "/ictstr01/groups/itg/shared/referenceData/OpenTargets/data_version_29_07"
output_path = main_dir / "temp/01_filtered_parquets"
manifest_path = main_dir / "temp/columns_manifest.json"

output_path.mkdir(parents=True, exist_ok=True)

# Load the columns manifest to get the list of datasets and their columns
with manifest_path.open() as f:
    manifest = json.load(f)

### Batch Processing of OpenTargets Datasets with Memory-Efficient Filtering

In [46]:
manifest.keys()

dict_keys(['colocalisation_coloc', 'credible_set', 'l2g_prediction', 'evidence', 'known_drug', 'disease', 'study', 'target', 'drug_indication', 'biosample_core'])

In [47]:
print(f"📋 Process {len(manifest)} datasets")

# Process datasets one by one to avoid memory issues
success_count = 0
failed_datasets = []

# Process large datasets first (they're most likely to cause memory issues)
large_datasets = ['colocalisation_coloc', 'credible_set']
small_datasets = [name for name in manifest.keys() if name not in large_datasets]

# Order: large datasets first, then small ones
processing_order = large_datasets + small_datasets

for dataset_name in processing_order:
    if dataset_name in manifest:
        columns = manifest[dataset_name]
        
        print(f"\n{'='*60}")
        print(f"Processing {dataset_name} ({success_count + 1}/{len(manifest)})")
        print(f"{'='*60}")
        
        success = process_single_dataset(dataset_name, columns, base_path, output_path)
        
        if success:
            success_count += 1
        else:
            failed_datasets.append(dataset_name)
        
        # Force garbage collection between datasets
        gc.collect()
        
        print(f"Memory cleanup completed.")

print(f"\n🎉 Processing complete!")
print(f"✅ Successfully processed: {success_count}/{len(manifest)} datasets")

if failed_datasets:
    print(f"❌ Failed datasets: {failed_datasets}")

print(f"📂 Filtered datasets saved to: {output_path}")

📋 Process 10 datasets

Processing colocalisation_coloc (1/10)

🔄 Processing colocalisation_coloc...
📋 Available columns (12): ['leftStudyLocusId', 'rightStudyLocusId', 'chromosome', 'rightStudyType', 'numberColocalisingVariants', 'h0', 'h1', 'h2', 'h3', 'h4']...
🔄 Converting to table with 10 columns: ['leftStudyLocusId', 'rightStudyLocusId', 'chromosome', 'h4', 'rightStudyType', 'numberColocalisingVariants', 'h0', 'h1', 'h2', 'h3']
📊 Large dataset detected, using batch processing...
  📊 Processed 10 batches, 5,626,147 rows so far...
  📊 Processed 20 batches, 10,630,414 rows so far...
  📊 Processed 30 batches, 15,424,575 rows so far...
  📊 Processed 40 batches, 21,052,744 rows so far...
  📊 Processed 50 batches, 26,057,870 rows so far...
  📊 Processed 60 batches, 30,844,974 rows so far...
  📊 Processed 70 batches, 36,474,946 rows so far...
✅ Saved colocalisation_coloc: 38,561,709 rows, 10 columns (38.1s)
Memory cleanup completed.

Processing credible_set (2/10)

🔄 Processing credible_se

In [8]:
df_final_available_doubleJoin = analyze_gwas_colocalizations_with_available_columns_doubleJoin(output_path)

🔍 Checking what columns are actually available in your filtered datasets...
  📋 colocalisation_coloc: ['leftStudyLocusId', 'rightStudyLocusId', 'chromosome', 'h4', 'rightStudyType', 'numberColocalisingVariants', 'h0', 'h1', 'h2', 'h3']
  📋 credible_set: ['studyLocusId', 'studyId', 'variantId', 'isTransQtl']
  📋 study: ['studyId', 'studyType', 'traitFromSource', 'projectId', 'pubmedId', 'geneId', 'biosampleFromSourceId', 'biosampleId']

🚀 Executing query with 20 available columns...
📋 Selected columns: ['leftStudyLocusId', 'rightStudyLocusId', 'studyId AS studyId_right', 'studyId AS studyId_left', 'chromosome', 'h0', 'h1', 'h2', 'h3', 'h4', 'traitFromSource AS traitFromSource_right', 'traitFromSource AS traitFromSource_left', 'studyType AS studyType_right', 'studyType AS studyType_left', 'numberColocalisingVariants', 'variantId AS variantId_right', 'variantId AS variantId_left', 'isTransQtl AS isTransQtl_right', 'isTransQtl AS isTransQtl_left', 'rightStudyType']


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Dataset created: 100 rows
📋 Final columns: ['leftStudyLocusId', 'rightStudyLocusId', 'studyId_right', 'studyId_left', 'chromosome', 'h0', 'h1', 'h2', 'h3', 'h4', 'traitFromSource_right', 'traitFromSource_left', 'studyType_right', 'studyType_left', 'numberColocalisingVariants', 'variantId_right', 'variantId_left', 'isTransQtl_right', 'isTransQtl_left', 'rightStudyType']

📊 Data completeness:
  leftStudyLocusId: 100/100 (100.0%)
  rightStudyLocusId: 100/100 (100.0%)
  studyId_right: 100/100 (100.0%)
  studyId_left: 100/100 (100.0%)
  chromosome: 100/100 (100.0%)
  h0: 100/100 (100.0%)
  h1: 100/100 (100.0%)
  h2: 100/100 (100.0%)
  h3: 100/100 (100.0%)
  h4: 100/100 (100.0%)
  traitFromSource_right: 100/100 (100.0%)
  traitFromSource_left: 100/100 (100.0%)
  studyType_right: 100/100 (100.0%)
  studyType_left: 100/100 (100.0%)
  numberColocalisingVariants: 100/100 (100.0%)
  variantId_right: 100/100 (100.0%)
  variantId_left: 100/100 (100.0%)
  isTransQtl_right: 100/100 (100.0%)
  isTra

In [42]:
df_final_available_doubleJoin

,leftStudyLocusId,rightStudyLocusId,studyId_right,studyId_left,chromosome,h0,h1,h2,h3,h4,traitFromSource_right,traitFromSource_left,studyType_right,studyType_left,numberColocalisingVariants,variantId_right,variantId_left,isTransQtl_right,isTransQtl_left,rightStudyType
0,73c2bc94afc604486d7ed065577877f0,19812ef02963fdc66ff3c1fd22d9f2d8,UKB_PPP_EUR_MFAP5_Q13361_OID20239_v1,GCST90278656,9,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,MFAP5:Q13361:OID20239:v1,Creatinine levels,pqtl,gwas,1,9_133274414_A_G,9_133274414_A_G,True,<NA>,pqtl
1,f05e95ca63a7b7ceee0dfb71f1d26aed,e7bf778b4b8a9fbef9182bd6b96fa9db,UKB_PPP_EUR_TLR3_O15455_OID20612_v1,GCST90309362,9,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,TLR3:O15455:OID20612:v1,Diabetes (confirmatory factor analysis Factor 28),pqtl,gwas,1,9_133274414_A_G,9_133274414_A_G,True,<NA>,pqtl
2,e622a8f68d69c67943caddee03021672,01ba8aabce3152b8647022fc925f5f9d,UKB_PPP_EUR_SERPINA4_P29622_OID30774_v1,GCST90445971,2,0.000000e+00,3.530834e-25,0.000000e+00,0.000000e+00,1.0,SERPINA4:P29622:OID30774:v1,Free cholesterol in medium LDL (UKB data field...,pqtl,gwas,1,2_27508073_T_C,2_27508073_T_C,True,<NA>,pqtl
3,b12b0486f0ccb8a6699439401593bf90,dee0f37c7376c67deb31e4e86877c59a,blueprint_tx_neutrophil_enst00000531313,GCST90018729,6,0.000000e+00,4.395203e-68,0.000000e+00,0.000000e+00,1.0,ENST00000531313,Weight,eqtl,gwas,1,6_130053316_T_A,6_130053316_T_A,False,<NA>,eqtl
4,c1003b3d4f00cd7e30cac8e129f1d1d7,e6a5c36e1ce60234ed22f0ed57c61729,UKB_PPP_EUR_IL1R2_P27930_OID20757_v1,GCST90445988,17,2.630247e-53,2.115630e-23,1.243245e-33,2.630247e-61,1.0,IL1R2:P27930:OID20757:v1,Free cholesterol in medium VLDL (UKB data fiel...,pqtl,gwas,1,17_7166093_G_A,17_7166093_G_A,True,<NA>,pqtl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9fccf1f77603fd2de0f19369e2cecb34,1f8f53d92e48b6b8eb0d1f550503ea5e,gtex_txrev_lung_ensg00000196628_grp_2_upstream...,GCST90309342,18,5.273720e-16,5.028360e-08,1.048795e-11,5.273720e-24,1.0,ENSG00000196628.grp_2.upstream.ENSG00000196628...,BMI and adiposity (confirmatory factor analysi...,tuqtl,gwas,1,18_55543071_G_T,18_55543071_G_T,False,<NA>,tuqtl
96,27ea1b8d635fb766aeab736856836cab,c2e063a2152ca0719c408a384db10123,UKB_PPP_EUR_ITGA5_P08648_OID21139_v1,GCST90302112,17,2.962949e-30,5.838812e-26,5.074575e-08,2.962949e-38,1.0,ITGA5:P08648:OID21139:v1,Phospholipids in small VLDL,pqtl,gwas,1,17_7166093_G_A,17_7166093_G_A,True,<NA>,pqtl
97,1dcc6c3ad6748ec7d827b96835e6a823,f583fd5b1953a895cbb5e5100475cfc2,UKB_PPP_EUR_NUMB_P49757_OID30568_v1,GCST90200738,3,1.264536e-26,7.803843e-23,1.620401e-07,1.264536e-34,1.0,NUMB:P49757:OID30568:v1,Adenosine 5'-monophosphate (AMP) to inosine 5'...,pqtl,gwas,1,3_56815721_T_C,3_56815721_T_C,True,<NA>,pqtl
98,030357ec0d2cf5bb72816feca4aa3868,956373d7a0ae3986e6ded162ebb56873,gtex_tx_adipose_visceral_enst00000476978,GCST90239652,1,2.822137e-67,9.761197e-64,2.891178e-07,2.822137e-75,1.0,ENST00000476978,High density lipoprotein cholesterol levels,eqtl,gwas,1,1_23383982_C_T,1_23383982_C_T,False,<NA>,eqtl


Adressed comments from Mauro:
- Affected gene mapped to the right study - ***We can get this from GWAS Study and merge on studyId to get geneID. Then,we can get the approvedSymbol from Target by merging the geneID on id***
- Affected tissue/cell mapped to the right study - ***This can be retreived from the studyId_right***

In [9]:
### join with target and GWAS datasets
target = pd.read_parquet(main_dir / output_path / 'target.parquet')
#study = pd.read_parquet('/lustre/groups/itg/shared/referenceData/OpenTargets/temp/01_filtered_parquets/study.parquet') 
study = pd.read_parquet(main_dir / output_path / 'study.parquet') 
biosample = pd.read_parquet(main_dir / output_path / 'biosample_core.parquet') 

# Merge study with df_final_available_doubleJoin by projectId (study) and studyId_left (df_final_available_doubleJoin)
coloc_study_merged = df_final_available_doubleJoin.merge(
    study,
    left_on='studyId_right',
    right_on='studyId',
    how='left',
    suffixes=('', '_study')
)


coloc_study_target_merged = coloc_study_merged.merge(
    target,
    left_on='geneId',
    right_on='id',
    how='left',
    suffixes=('', '_target')
)

# Define columns to drop (these are duplicates after merge)
columns_to_drop = [
    'studyId',        # duplicate of studyId_right or studyId_left
    'id',             # duplicate of geneId
    'biosampleFromSourceId', # duplicate if already present
    'biotype'         # duplicate if already present
]

coloc_study_target_merged_clean = coloc_study_target_merged.drop(columns=columns_to_drop)

coloc_study_target_merged_clean = coloc_study_target_merged_clean.rename(columns={
    'studyId_right': 'studyId',
    'geneId': 'geneId',
    'approvedSymbol': 'gene_symbol',
    'biotype': 'gene_biotype'
})

# Add biosample information
coloc_study_target_merged_clean = coloc_study_target_merged_clean.merge(
    biosample,
    left_on='biosampleId',
    right_on='biosampleId',
    how='left',
    suffixes=('', '_biosample_core')
)


pd.set_option('display.max_columns', None)
coloc_study_target_merged_clean

,leftStudyLocusId,rightStudyLocusId,studyId,studyId_left,chromosome,h0,h1,h2,h3,h4,traitFromSource_right,traitFromSource_left,studyType_right,studyType_left,numberColocalisingVariants,variantId_right,variantId_left,isTransQtl_right,isTransQtl_left,rightStudyType,studyType,traitFromSource,projectId,pubmedId,geneId,biosampleId,gene_symbol,biosampleName
0,5f17171742cb336e598058a792057818,c823b07af29ab802304808ae6a8c6395,UKB_PPP_EUR_CDH1_P12830_OID20353_v1,GCST90445869,17,9.959958e-39,9.458834e-17,1.052980e-25,9.959958e-47,1.0,CDH1:P12830:OID20353:v1,Cholesteryl esters in IDL (UKB data field 23527),pqtl,gwas,1,17_7166093_G_A,17_7166093_G_A,True,<NA>,pqtl,pqtl,CDH1:P12830:OID20353:v1,UKB_PPP_EUR,None,ENSG00000039068,UBERON_0001969,CDH1,blood plasma
1,0882eff81348a20da2f1c6bd2755a03c,a272b836aa06c97898d6e2930dcd002c,UKB_PPP_EUR_ICOSLG_O75144_OID21351_v1,GCST90445938,17,4.311770e-72,5.303356e-54,8.130267e-22,4.311770e-80,1.0,ICOSLG:O75144:OID21351:v1,Cholesterol in large VLDL (UKB data field 23498),pqtl,gwas,1,17_7166093_G_A,17_7166093_G_A,True,<NA>,pqtl,pqtl,ICOSLG:O75144:OID21351:v1,UKB_PPP_EUR,None,ENSG00000160223,UBERON_0001969,ICOSLG,blood plasma
2,25aae3f7d879957a615544101313e94c,d254d723672473afc5663dc1ea417c6d,UKB_PPP_EUR_PTH_P01270_OID30963_v1,GCST90302116,19,0.000000e+00,1.911257e-241,5.594850e-241,0.000000e+00,1.0,PTH:P01270:OID30963:v1,Ratio of triglycerides to phosphoglycerides,pqtl,gwas,1,19_44885967_G_A,19_44885967_G_A,True,<NA>,pqtl,pqtl,PTH:P01270:OID30963:v1,UKB_PPP_EUR,None,ENSG00000152266,UBERON_0001969,PTH,blood plasma
3,87aea904ffddabb7eb00f6567223b687,c6e31b8d12f0a63e3a054994dd59b01e,UKB_PPP_EUR_CA12_O43570_OID21344_v1,GCST90475252,4,2.748606e-51,3.114083e-18,8.826374e-37,2.748606e-59,1.0,CA12:O43570:OID21344:v1,"diastolic blood pressure (DBP, maximum, inv-no...",pqtl,gwas,1,4_102267552_C_T,4_102267552_C_T,True,<NA>,pqtl,pqtl,CA12:O43570:OID21344:v1,UKB_PPP_EUR,None,ENSG00000074410,UBERON_0001969,CA12,blood plasma
4,7938f4902f1f71612385eb0966f51127,de2935ab82010a3e38e29841f10ab1ed,UKB_PPP_EUR_TNFRSF10C_O14798_OID20265_v1,GCST90013984,7,4.319568e-149,3.893943e-22,1.109304e-130,4.319568e-157,1.0,TNFRSF10C:O14798:OID20265:v1,Neutrophill count (UKB data field 30140),pqtl,gwas,1,7_92779056_C_T,7_92779056_C_T,True,<NA>,pqtl,pqtl,TNFRSF10C:O14798:OID20265:v1,UKB_PPP_EUR,None,ENSG00000173535,UBERON_0001969,TNFRSF10C,blood plasma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,a47966a1eff8dfdcc13d61d988a6d05e,94e37c1eee2ce38cca1674d86cac81e6,UKB_PPP_EUR_MAMDC2_Q7Z304_OID30309_v1,GCST90003775,4,6.479199e-49,3.891017e-42,1.665169e-10,6.479199e-57,1.0,MAMDC2:Q7Z304:OID30309:v1,aseg lh intensity VentralDC,pqtl,gwas,1,4_102267552_C_T,4_102267552_C_T,True,<NA>,pqtl,pqtl,MAMDC2:Q7Z304:OID30309:v1,UKB_PPP_EUR,None,ENSG00000165072,UBERON_0001969,MAMDC2,blood plasma
96,04088a32f6d6a662d30a54cd8c335618,8fb5094a447d4949ac3300d7c662e5a6,UKB_PPP_EUR_NFX1_Q12986_OID30158_v1,GCST90200374,3,1.134966e-17,1.581191e-10,7.177923e-11,1.134966e-25,1.0,NFX1:Q12986:OID30158:v1,Sphinganine levels,pqtl,gwas,1,3_56815721_T_C,3_56815721_T_C,True,<NA>,pqtl,pqtl,NFX1:Q12986:OID30158:v1,UKB_PPP_EUR,None,ENSG00000086102,UBERON_0001969,NFX1,blood plasma
97,f01167820a5fd0b0fe5e3c66edcbdd64,a38409f30e58eaacd22894e8b5af4d01,UKB_PPP_EUR_GFRAL_Q6UXV0_OID31091_v1,GCST90269603,17,3.964613e-20,1.625766e-13,2.438613e-10,3.964613e-28,1.0,GFRAL:Q6UXV0:OID31091:v1,Cholesteryl esters in medium VLDL (UKB data fi...,pqtl,gwas,1,17_7166093_G_A,17_7166093_G_A,True,<NA>,pqtl,pqtl,GFRAL:Q6UXV0:OID31091:v1,UKB_PPP_EUR,None,ENSG00000187871,UBERON_0001969,GFRAL,blood plasma
98,d4dbd8486c35b0814fcaa958f812ab54,1f37a335058b3655e8f15f5dc8eab90b,UKB_PPP_EUR_CD4_P01730_OID20584_v1,GCST90132670,11,7.917587e-26,2.581091e-19,3.067535e-10,7.917587e-34,1.0,CD4:P01730:OID20584:v1,Cholesterol ester levels in large HDL,pqtl,gwas,1,11_116778201_G_C,11_116778201_G_C,True,<NA>,pqtl,pqtl,CD4:P01730:OID20584:v1,UKB_PPP_EUR

Add tissue name

In [52]:
coloc_study_target_merged_clean

,leftStudyLocusId,rightStudyLocusId,studyId,studyId_left,chromosome,h0,h1,h2,h3,h4,traitFromSource_right,traitFromSource_left,studyType_right,studyType_left,numberColocalisingVariants,variantId_right,variantId_left,isTransQtl_right,isTransQtl_left,rightStudyType,studyType,traitFromSource,projectId,pubmedId,geneId,biosampleId,gene_symbol,biosampleName
0,73c2bc94afc604486d7ed065577877f0,19812ef02963fdc66ff3c1fd22d9f2d8,UKB_PPP_EUR_MFAP5_Q13361_OID20239_v1,GCST90278656,9,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,MFAP5:Q13361:OID20239:v1,Creatinine levels,pqtl,gwas,1,9_133274414_A_G,9_133274414_A_G,True,<NA>,pqtl,pqtl,MFAP5:Q13361:OID20239:v1,UKB_PPP_EUR,None,ENSG00000197614,UBERON_0001969,MFAP5,blood plasma
1,f05e95ca63a7b7ceee0dfb71f1d26aed,e7bf778b4b8a9fbef9182bd6b96fa9db,UKB_PPP_EUR_TLR3_O15455_OID20612_v1,GCST90309362,9,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,TLR3:O15455:OID20612:v1,Diabetes (confirmatory factor analysis Factor 28),pqtl,gwas,1,9_133274414_A_G,9_133274414_A_G,True,<NA>,pqtl,pqtl,TLR3:O15455:OID20612:v1,UKB_PPP_EUR,None,ENSG00000164342,UBERON_0001969,TLR3,blood plasma
2,e622a8f68d69c67943caddee03021672,01ba8aabce3152b8647022fc925f5f9d,UKB_PPP_EUR_SERPINA4_P29622_OID30774_v1,GCST90445971,2,0.000000e+00,3.530834e-25,0.000000e+00,0.000000e+00,1.0,SERPINA4:P29622:OID30774:v1,Free cholesterol in medium LDL (UKB data field...,pqtl,gwas,1,2_27508073_T_C,2_27508073_T_C,True,<NA>,pqtl,pqtl,SERPINA4:P29622:OID30774:v1,UKB_PPP_EUR,None,ENSG00000100665,UBERON_0001969,SERPINA4,blood plasma
3,b12b0486f0ccb8a6699439401593bf90,dee0f37c7376c67deb31e4e86877c59a,blueprint_tx_neutrophil_enst00000531313,GCST90018729,6,0.000000e+00,4.395203e-68,0.000000e+00,0.000000e+00,1.0,ENST00000531313,Weight,eqtl,gwas,1,6_130053316_T_A,6_130053316_T_A,False,<NA>,eqtl,eqtl,ENST00000531313,BLUEPRINT,27863251,ENSG00000198945,CL_0000775,L3MBTL3,neutrophil
4,c1003b3d4f00cd7e30cac8e129f1d1d7,e6a5c36e1ce60234ed22f0ed57c61729,UKB_PPP_EUR_IL1R2_P27930_OID20757_v1,GCST90445988,17,2.630247e-53,2.115630e-23,1.243245e-33,2.630247e-61,1.0,IL1R2:P27930:OID20757:v1,Free cholesterol in medium VLDL (UKB data fiel...,pqtl,gwas,1,17_7166093_G_A,17_7166093_G_A,True,<NA>,pqtl,pqtl,IL1R2:P27930:OID20757:v1,UKB_PPP_EUR,None,ENSG00000115590,UBERON_0001969,IL1R2,blood plasma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9fccf1f77603fd2de0f19369e2cecb34,1f8f53d92e48b6b8eb0d1f550503ea5e,gtex_txrev_lung_ensg00000196628_grp_2_upstream...,GCST90309342,18,5.273720e-16,5.028360e-08,1.048795e-11,5.273720e-24,1.0,ENSG00000196628.grp_2.upstream.ENSG00000196628...,BMI and adiposity (confirmatory factor analysi...,tuqtl,gwas,1,18_55543071_G_T,18_55543071_G_T,False,<NA>,tuqtl,tuqtl,ENSG00000196628.grp_2.upstream.ENSG00000196628...,GTEx,32913098,ENSG00000196628,UBERON_0008952,TCF4,upper lobe of left lung
96,27ea1b8d635fb766aeab736856836cab,c2e063a2152ca0719c408a384db10123,UKB_PPP_EUR_ITGA5_P08648_OID21139_v1,GCST90302112,17,2.962949e-30,5.838812e-26,5.074575e-08,2.962949e-38,1.0,ITGA5:P08648:OID21139:v1,Phospholipids in small VLDL,pqtl,gwas,1,17_7166093_G_A,17_7166093_G_A,True,<NA>,pqtl,pqtl,ITGA5:P08648:OID21139:v1,UKB_PPP_EUR,None,ENSG00000161638,UBERON_0001969,ITGA5,blood plasma
97,1dcc6c3ad6748ec7d827b96835e6a823,f583fd5b1953a895cbb5e5100475cfc2,UKB_PPP_EUR_NUMB_P49757_OID30568_v1,GCST90200738,3,1.264536e-26,7.803843e-23,1.620401e-07,1.264536e-34,1.0,NUMB:P49757:OID30568:v1,Adenosine 5'-monophosphate (AMP) to inosine 5'...,pqtl,gwas,1,3_56815721_T_C,3_56815721_T_C,True,<NA>,pqtl,pqtl,NUMB:P49757:OID30568:v1,UKB_PPP_EUR,None,ENSG00000133961,UBERON_0001969,NUMB,blood plasma
98,030357ec0d2cf5bb72816feca4aa3868,956373d7a0ae3986e6ded162ebb56873,gtex_tx_adipose_visceral_enst00000476978,GCST90239652,1,2.822137e-67,9.761197e-64,2.891178e-07,2.822137e-75,1.0,ENST00000476978,High density lipoprotein cholesterol levels,eqtl,gwas,1,1_23383982_C_T,1_23383982_C_T,False,<NA>,eqtl,eqtl,ENST0000

In [32]:
coloc_study_target_merged_clean.columns

Index(['leftStudyLocusId', 'rightStudyLocusId', 'studyId', 'studyId_left',
       'chromosome', 'h0', 'h1', 'h2', 'h3', 'h4', 'traitFromSource_right',
       'traitFromSource_left', 'studyType_right', 'studyType_left',
       'numberColocalisingVariants', 'variantId_right', 'variantId_left',
       'isTransQtl_right', 'isTransQtl_left', 'rightStudyType', 'studyType',
       'traitFromSource', 'projectId', 'pubmedId', 'geneId'],
      dtype='object')

### Colocalization Dataset Generation

In [11]:
print("🚀 Creating dataset with available columns including numberColocalisingVariants, variantId, and isTransQtl...")
df_final_available = analyze_gwas_colocalizations_with_available_columns(output_path)

if df_final_available is not None:
    print(f"\n🎉 Analysis complete!")
    print(f"📊 Generated {len(df_final_available)} rows with available columns")
    
    # Save the dataset
    available_file = main_dir / "colocalisation_available_columns_with_variants.csv"
    df_final_available.to_csv(available_file, index=False)
    print(f"💾 Saved to: {available_file}")
    
    # Show what you got vs what you wanted
    print(f"\n📋 Comparison - Requested vs Available:")
    requested = ['leftStudyLocusId','rightStudyLocusId', 'studyId', 'traitFromSource', 'chromosome', 
                'studyType', 'rightStudyType', 'numberColocalisingVariants', 'variantId',
                'isTransQtl', 'h0', 'h1', 'h2', 'h3', 'h4']
    
    available_cols = []
    missing_cols = []
    for col in requested:
        if col in df_final_available.columns:
            available_cols.append(col)
            print(f"  ✅ {col}")
        else:
            missing_cols.append(col)
            print(f"  ❌ {col} (not in your filtered dataset)")
    
    print(f"\n📊 Summary:")
    print(f"  ✅ Available columns: {len(available_cols)}/{len(requested)} ({len(available_cols)/len(requested)*100:.1f}%)")
    print(f"  📋 Available: {available_cols}")
    if missing_cols:
        print(f"  ❌ Missing: {missing_cols}")
    
    print(f"\n💡 To get missing columns:")
    print(f"  • Add isTransQtl to your manifest's credible_set columns")
    print(f"  • Re-run the dataset filtering process")
    print(f"  • Then run this analysis to get the complete dataset")
    
else:
    print("❌ Failed to create dataset")

🚀 Creating dataset with available columns including numberColocalisingVariants, variantId, and isTransQtl...
🔍 Checking what columns are actually available in your filtered datasets...
  📋 colocalisation_coloc: ['leftStudyLocusId', 'rightStudyLocusId', 'chromosome', 'h4', 'rightStudyType', 'numberColocalisingVariants', 'h0', 'h1', 'h2', 'h3']
  📋 credible_set: ['studyLocusId', 'studyId', 'variantId', 'isTransQtl']
  📋 study: ['studyId', 'studyType', 'traitFromSource', 'projectId', 'pubmedId']

🔧 Building query with available columns...
  ✅ Available h-columns: ['h0', 'h1', 'h2', 'h3', 'h4']
  ✅ Available study columns: ['traitFromSource', 'studyType']
  ✅ numberColocalisingVariants: Available
  ✅ variantId: Available
  ✅ isTransQtl: Available
  ✅ rightStudyType: Available

🚀 Executing query with 15 available columns...
📋 Selected columns: ['leftStudyLocusId', 'rightStudyLocusId', 'studyId', 'chromosome', 'h0', 'h1', 'h2', 'h3', 'h4', 'traitFromSource', 'studyType', 'numberColocalisingV

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Dataset created: 100 rows
📋 Final columns: ['leftStudyLocusId', 'rightStudyLocusId', 'studyId', 'chromosome', 'h0', 'h1', 'h2', 'h3', 'h4', 'traitFromSource', 'studyType', 'numberColocalisingVariants', 'variantId', 'isTransQtl', 'rightStudyType']

📊 Data completeness:
  leftStudyLocusId: 100/100 (100.0%)
  rightStudyLocusId: 100/100 (100.0%)
  studyId: 100/100 (100.0%)
  chromosome: 100/100 (100.0%)
  h0: 100/100 (100.0%)
  h1: 100/100 (100.0%)
  h2: 100/100 (100.0%)
  h3: 100/100 (100.0%)
  h4: 100/100 (100.0%)
  traitFromSource: 100/100 (100.0%)
  studyType: 100/100 (100.0%)
  numberColocalisingVariants: 100/100 (100.0%)
  variantId: 100/100 (100.0%)
  isTransQtl: 11/100 (11.0%)
  rightStudyType: 100/100 (100.0%)

👀 Sample of your dataset:
                   leftStudyLocusId                 rightStudyLocusId  \
0  8c3672c9822525cc91ec0e3b164ec7ec  363d740259e5ef96863c6690bf99f42e   
1  b27d089888c12f39398df55581513e3f  48597ac988b6a51483baee8405eb3028   
2  e792964bda81a15aae6c989f

### Gene-Drug-Target Association Analysis and Summary

In [19]:
df_gene_target = run_gene_drug_analysis()

# Remove columns that are not needed
df_gene_target = df_gene_target.drop(columns=["studyLocusId","l2g_score"])

df_gene_target = df_gene_target.drop_duplicates()

if df_gene_target is not None:
    print("\n📊 Gene-Drug-Target Data Analysis:")
    print(f"  Total associations: {len(df_gene_target):,}")
    print(f"  Unique genes: {df_gene_target['geneId'].nunique():,}")
    print(f"  Unique drugs: {df_gene_target['drugId'].nunique():,}")
    print(f"  Unique targets: {df_gene_target['targetId'].nunique():,}")
    print(f"  Unique diseases: {df_gene_target['diseaseId'].nunique():,}")

    # Top genes by drug count
    top_genes = df_gene_target.groupby('gene_symbol')['drugId'].nunique().sort_values(ascending=False).head(5)
    print("\n🔝 Top genes by drug count:")
    for gene, count in top_genes.items():
        print(f"    {gene}: {count} drugs")

    # L2G score distribution
    #print("\n🎯 L2G Score Statistics:")
    #print(df_gene_target['l2g_score'].describe())

    # Drug development phase analysis (if available)
    if 'phase' in df_gene_target.columns:
        print("\n💊 Drug Development Phases:")
        print(df_gene_target['phase'].value_counts().sort_index())

    # Biotype distribution
    print("\n🧬 Biotype Distribution:")
    print(df_gene_target['biotype'].value_counts())

    # Show sample data
    print("\n👀 Sample associations:")
    print(df_gene_target.head())
else:
    print("❌ No gene-drug-target associations found.")

🚀 Creating gene-drug-target dataframe...
🔍 Checking available datasets for gene-drug-target mapping...
  📋 l2g_prediction: ['studyLocusId', 'geneId', 'score']
  📋 evidence: ['studyId', 'targetId', 'diseaseId']
  📋 known_drug: ['targetId', 'diseaseId', 'drugId']
  📋 target: ['id', 'approvedSymbol', 'biotype']

🔧 Building gene-drug-target query...
  ✅ l2g_prediction: geneId available
  ✅ evidence: targetId available
  ✅ known_drug: ['drugId'] available
  ✅ target: ['gene_symbol', 'biotype'] available

🚀 Executing gene-drug-target query...
📋 Selected columns: ['geneId', 'studyLocusId', 'l2g_score', 'targetId', 'diseaseId', 'drugId', 'gene_symbol', 'biotype']


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Dataset created: 1000 rows
📋 Final columns: ['geneId', 'studyLocusId', 'l2g_score', 'targetId', 'diseaseId', 'drugId', 'gene_symbol', 'biotype']

📊 Gene-Drug-Target Summary:
  Unique genes: 2
  Unique drugs: 9
  Unique targets: 2

🎯 Locus-to-Gene Score Distribution:
  Mean score: 1.000
  High confidence (>0.8): 1000 genes

👀 Sample gene-drug-target associations:
            geneId gene_symbol         drugId         targetId  l2g_score
0  ENSG00000151079       KCNA6   CHEMBL284348  ENSG00000151079   0.999812
1  ENSG00000151079       KCNA6   CHEMBL354077  ENSG00000151079   0.999812
2  ENSG00000151079       KCNA6   CHEMBL354077  ENSG00000151079   0.999812
3  ENSG00000151079       KCNA6   CHEMBL284348  ENSG00000151079   0.999812
4  ENSG00000151079       KCNA6  CHEMBL1200728  ENSG00000151079   0.999812

🎉 Analysis complete!
📊 Generated 1000 gene-drug-target associations
💾 Saved to: /ictstr01/groups/itg/shared/referenceData/OpenTargets/Colocalizations/envs/gene_drug_target_associations.csv

In [13]:
drug_indication = pd.read_parquet(main_dir / output_path / 'drug_indication.parquet') 
drug_indication

,id,indications,approvedIndications
0,CHEMBL1000,"[{'disease': 'EFO_0008521', 'efoName': 'rhinit...","[EFO_0005854, EFO_0005531, HP_0000989, MONDO_0..."
1,CHEMBL100014,"[{'disease': 'EFO_0003060', 'efoName': 'non-sm...",[]
2,CHEMBL100116,"[{'disease': 'MONDO_0004985', 'efoName': 'bipo...",[EFO_0003843]
3,CHEMBL1002,"[{'disease': 'MONDO_0004979', 'efoName': 'asth...",[MONDO_0004979]
4,CHEMBL100259,"[{'disease': 'MONDO_0008903', 'efoName': 'lung...",[]
...,...,...,...
9673,CHEMBL9960,"[{'disease': 'MONDO_0004979', 'efoName': 'asth...",[]
9674,CHEMBL9967,"[{'disease': 'EFO_0003948', 'efoName': 'gastro...","[EFO_0003948, HP_0004398]"
9675,CHEMBL997,"[{'disease': 'HP_0002027', 'efoName': 'abdomin...","[EFO_0003854, EFO_0003882]"
9676,CHEMBL998,"[{'disease': 'EFO_0004232', 'efoName': 'eosino...","[EFO_0005854, EFO_0005531, HP_0000989, MONDO_0..."


In [14]:
# explode the 'indications' column (list of dicts) into separate rows
df_long = drug_indication.explode("indications").reset_index(drop=True)

# turn the dict in 'indications' into columns
indications_expanded = pd.json_normalize(df_long["indications"])
indications_expanded["id"] = df_long["id"].values

# join back
indications_expanded = df_long.drop(columns="approvedIndications").merge(indications_expanded, on="id")

# clean-up
indications_expanded = indications_expanded.drop(columns=["indications", 'references'])

indications_expanded.head()

,id,disease,efoName,maxPhaseForIndication
0,CHEMBL1000,EFO_0008521,rhinitis,2.0
1,CHEMBL1000,HP_0000989,pruritus,4.0
2,CHEMBL1000,EFO_0004256,neuromyelitis optica,1.0
3,CHEMBL1000,EFO_1001417,"rhinitis, allergic, perennial",3.0
4,CHEMBL1000,MONDO_0005271,allergic disease,1.0


In [24]:
df_gene_target_with_indications = df_gene_target.merge(
    indications_expanded,
    left_on = ['drugId','diseaseId'],
    right_on = ['id','disease'],
)

df_gene_target_with_indications = df_gene_target_with_indications.drop(columns=["disease","id"])

df_gene_target_with_indications = df_gene_target_with_indications.drop_duplicates()

df_gene_target_with_indications
 

,geneId,targetId,diseaseId,drugId,gene_symbol,biotype,efoName,maxPhaseForIndication
0,ENSG00000151079,ENSG00000151079,MONDO_0005301,CHEMBL284348,KCNA6,protein_coding,multiple sclerosis,4.0
17,ENSG00000151079,ENSG00000151079,MONDO_0005301,CHEMBL354077,KCNA6,protein_coding,multiple sclerosis,3.0
28,ENSG00000151079,ENSG00000151079,EFO_0004991,CHEMBL1200728,KCNA6,protein_coding,myasthenia gravis,4.0
31,ENSG00000151079,ENSG00000151079,MONDO_0005301,CHEMBL2107762,KCNA6,protein_coding,multiple sclerosis,2.0
33,ENSG00000151079,ENSG00000151079,EFO_0003782,CHEMBL284348,KCNA6,protein_coding,motor neuron disease,1.0
50,ENSG00000151079,ENSG00000151079,EFO_0003911,CHEMBL2111110,KCNA6,protein_coding,atrial flutter,3.0
53,ENSG00000151079,ENSG00000151079,EFO_0004991,CHEMBL3301611,KCNA6,protein_coding,myasthenia gravis,3.0
54,ENSG00000151079,ENSG00000151079,EFO_0004991,CHEMBL3301611,KCNA6,protein_coding,myasthenia gravis,4.0
65,ENSG00000151079,ENSG00000151079,EFO_0004991,CHEMBL354077,KCNA6,protein_coding,myasthenia gravis,4.0
66,ENSG00000151079,ENSG00000151079,EFO_0004991,CHEMBL354077,KCNA6,protein_coding,myasthenia gravis,3.0
